In [ ]:
#### traning model with CNN model

In [1]:
import tensorflow as tf
from tensorflow import keras
import PIL
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import json
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image

from tensorflow.keras import Sequential

In [2]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'Fire-vs-NoFire\\Training', validation_split=0.2, subset="training",seed=1337,batch_size=32,shuffle=True
)

Found 39375 files belonging to 2 classes.
Using 31500 files for training.


In [3]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'Fire-vs-NoFire\\Training', validation_split=0.2, subset="validation", seed=1337,
        batch_size=32, shuffle=True
        )

Found 39375 files belonging to 2 classes.
Using 7875 files for validation.


In [4]:
image_size = (256,256)

In [6]:
def make_model_CNN(input_shape, num_classes):

    print(input_shape)
   
    inputs = keras.Input(shape=input_shape)
    
    x = inputs  

    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    
    x = layers.Conv2D(3, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    x = layers.Conv2D(3, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    
    x = layers.Conv2D(3, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
  
    x = layers.GlobalAveragePooling2D()(x)
    #x = layers.Dense(2,activation='relu')
    
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes
    
    x = layers.Dropout(0.5)(x)
    
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs, name="model_cnn")

In [8]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

model = make_model_CNN(input_shape=image_size + (3,), num_classes=2)

model.summary()


(256, 256, 3)
Model: "model_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 3)       84        
_________________________________________________________________
batch_normalization_3 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
activation_3 (Activation)    (None, 128, 128, 3)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32,

In [9]:
checkpoint = [keras.callbacks.ModelCheckpoint(filepath="CNN_MODEL\\save_at_{epoch}.h5", verbose=1, save_best_only=True), ]
#otp = keras.optimizers.
model.compile(optimizer=keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.9), loss="binary_crossentropy", metrics=["accuracy"], )
model.fit(train_ds, epochs=40, callbacks=checkpoint, validation_data=val_ds, batch_size=32)

Epoch 1/40
984/985 [============================>.] - ETA: 0s - loss: 0.3963 - accuracy: 0.8611
Epoch 00001: val_loss improved from inf to 0.29925, saving model to CNN_MODEL\save_at_1.h5
985/985 [==============================] - 119s 121ms/step - loss: 0.3966 - accuracy: 0.8611 - val_loss: 0.2992 - val_accuracy: 0.9387
Epoch 2/40
984/985 [============================>.] - ETA: 0s - loss: 0.3618 - accuracy: 0.8685
Epoch 00002: val_loss improved from 0.29925 to 0.28071, saving model to CNN_MODEL\save_at_2.h5
985/985 [==============================] - 119s 121ms/step - loss: 0.3619 - accuracy: 0.8684 - val_loss: 0.2807 - val_accuracy: 0.9509
Epoch 3/40
984/985 [============================>.] - ETA: 0s - loss: 0.3405 - accuracy: 0.8776
Epoch 00003: val_loss improved from 0.28071 to 0.22777, saving model to CNN_MODEL\save_at_3.h5
985/985 [==============================] - 121s 123ms/step - loss: 0.3405 - accuracy: 0.8775 - val_loss: 0.2278 - val_accuracy: 0.9599
Epoch 4/40
984/985 [======

In [12]:
 model.save("CNN_MODEL\\model")

INFO:tensorflow:Assets written to: CNN_MODEL\model\assets
